In [53]:
%matplotlib inline

import tushare as ts
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff
import numpy as np
from collections import namedtuple



scatter_info = namedtuple('scatter_info', ['x_pos', 'y_pos', 'name', 'color', 'opacity'])
distplots_info = namedtuple('distplots_info', ['hist_data', 'group_labels', 'colors', 'title_name'])
bar_info = namedtuple('bar_info', ['x_pos', 'y_pos', 'text', 'name', 'fill_color', 'opacity'])

ts.set_token('a309ac73e318eb784409e517eca79159e855faf2b6a4df623cc33684')
pro = ts.pro_api()
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
def date_format_transfer(date, split_simbol='-'):
    return ["{}{}{}{}{}".format(a[:4], split_simbol, a[4:6], split_simbol, a[6:8]) for a in date]

In [ ]:
def get_day_of_date(date):
    return [a[6:8] for a in date]

In [ ]:
def get_month_daily_datum(pro, ts_code, year, month, fields, is_index=False):
    start_date = "{}{}01".format(year, month)
    end_date = "{}{}31".format(year, month)
    if is_index:
        return pro.index_daily(ts_code=ts_code, start_date=start_date, end_date=end_date, fields=fields)
    else:
        return pro.daily(ts_code=ts_code, start_date=start_date, end_date=end_date, fields=fields)

In [ ]:
def draw_scatter_diagram(data_infos):
    datum = [go.Scatter(
                x = info.x_pos,
                y = info.y_pos,
                name = info.name,
                line = dict(color = info.color),
                opacity = info.opacity) for info in data_infos]
    plotly.offline.iplot(datum)

In [ ]:
def draw_bar_diagram(data_infos, title=None):
    data = [go.Bar(
                x=data.x_pos,
                y=data.y_pos,
                text=data.text,
                textposition = 'auto',
                name=data.name,
                marker=dict(
                    color=data.fill_color,
                ),
                opacity=data.opacity
            ) for data in data_infos]
    layout = go.Layout(title=title,)
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig)

In [ ]:
def draw_distplots_diagram(data_infos, show_hist=True, show_curve=True, show_rug=True, title_name=None):
    fig = ff.create_distplot(data_infos.hist_data, data_infos.group_labels, colors=data_infos.colors,
                             show_hist=show_hist, show_curve=show_curve, show_rug=show_rug)
    fig['layout'].update(title=title_name)   
    plotly.offline.iplot(fig)

In [ ]:
fields = "trade_date, pct_chg"
# data = pro.daily(ts_code='002236.SZ', start_date='20180101', end_date='20181231', fields=fields)
data = pro.query('daily', ts_code='002236.SZ', start_date='20180101', end_date='20181231', fields=fields)
pctchg_info = distplots_info([data.pct_chg], ['pct chg'], ['#333F44'], 
                             title_name='pct change rate statistics of 2018 whole year')
draw_distplots_diagram(pctchg_info, show_rug=False)

In [ ]:
fields = "trade_date, pct_chg"
date9 = get_month_daily_datum(pro, '002236.SZ', '2018', '09', fields)
date9_info = scatter_info(get_day_of_date(date9.trade_date), date9.pct_chg, 'Sep', '#FF0000', 0.8)
date10 = get_month_daily_datum(pro, '002236.SZ', '2018', '10', fields)
date10_info = scatter_info(get_day_of_date(date10.trade_date), date10.pct_chg, 'Oct', '#00FF00', 0.8)
date11 = get_month_daily_datum(pro, '002236.SZ', '2018', '11', fields)
date11_info = scatter_info(get_day_of_date(date11.trade_date), date11.pct_chg, 'Nov', '#0000FF', 0.8)
date_infos = [date9_info, date10_info, date11_info] 
draw_scatter_diagram(date_infos)

In [ ]:
fields = "trade_date, pct_chg"
start_date = '20180101'
end_date = '20181231'
index_date = pro.index_daily(ts_code='399001.SZ', start_date=start_date, end_date=end_date, fields=fields)
index_date_info = scatter_info(date_format_transfer(index_date.trade_date), index_date.pct_chg, 'SZ_index', '#00FF00', 0.8)
stock_date = pro.daily(ts_code='002236.SZ', start_date=start_date, end_date=end_date, fields=fields)
stock_date_info = scatter_info(date_format_transfer(stock_date.trade_date), stock_date.pct_chg, 'Dahua', '#FF0000', 0.8)
date_infos = [stock_date_info, index_date_info] 
draw_scatter_diagram(date_infos)

In [ ]:
df = ts.pro_bar(pro_api=pro, ts_code='002236.SZ', start_date='20181120', end_date='20181128')
dfq = ts.pro_bar(pro_api=pro,ts_code='002236.SZ', adj='qfq', start_date='20181120', end_date='20181128')
dfh = ts.pro_bar(pro_api=pro,ts_code='002236.SZ', adj='hfq', start_date='20181120', end_date='20181128')
print(df)
print(dfq)
print(dfh)


In [ ]:
print(pro.pledge_stat(ts_code='002236.SZ'))


In [ ]:
fields = "trade_date, pct_chg"
data1 = pro.daily(ts_code='002236.SZ', start_date='20180101', end_date='20181231', fields=fields)

print(data1[data1.pct_chg>0].pct_chg.describe())
print(data1[data1.pct_chg<=0].pct_chg.describe())

In [ ]:
fields = "trade_date, pct_chg"
df_index = pro.index_daily(ts_code='399001.SZ', )
df

In [86]:
def get_ATR(data):
    return max(data[0]-data[1], abs(data[0]-data[2]), abs(data[2]-data[1]))

In [85]:
def get_N_Initial_Value(Initial_N_data):
    return np.array([get_ATR(x) for x in Initial_N_data.to_numpy()]).mean()

In [133]:
def get_N_value(data, total_days=40):
    N_0 = get_N_Initial_Value(data[total_days-20:total_days])
    N = PDN = N_0
    for x in reversed(data[:total_days-20].to_numpy()):
        N = (19*PDN + get_ATR(x)) / 20
        PDN = N
    print("N is", N)
    return N

In [157]:
def calculate_position_unit(account, N_value, U_data):
    position_unit = round((0.01 * account) / (N_value * 100 * U_data))
    print("position_unit is", position_unit)
    return position_unit


In [148]:
def calculate_market_in_and_out_price_with_long(data, period_in, period_out):
    return {"In_val": data[:period_in]['high'].max(), "Out_val": data[:period_out]['low'].min()}

In [158]:
data = pro.daily(ts_code="002236.SZ")
N_data = data[['high', 'low', 'pre_close']]
N_value = get_N_value(N_data, 100)

U_data = data.iloc[0]['close']
calculate_position_unit(50000, N_value, U_data)

BT_data = data[['high', 'low']]
calculate_market_in_and_out_price_with_long(BT_data, 20, 10)

N is 0.4484610527281235
position_unit is 1.0


{'In_val': 12.73, 'Out_val': 11.9}